Visualization Tool: dash.plotly

# To-Do
- [ ] Pagination 1a --> Tina
- [ ] Zweck der Daten --> Tina
- [ ] Vergleich zwischen Players 1d --> Huo
- [ ] Gegenüberstellung zwischen Age und Wage/Overall 1b --> Huo
- [ ] zusätzliche Filtern

# Aufgabe 1a)
Lesen Sie den Datensatz „Aufgabe-1.csv“ ein und untersuchen Sie den Zweck der Daten sowie die
Datenqualität. Identifizieren Sie mögliche Probleme in den Daten und beheben Sie die Fehler, falls
möglich. Dokumentieren Sie Ihr Vorgehen.  

In [ ]:
import pandas as pd

In [ ]:
#df = pd.read_csv('aufgabe1_ori.csv')

## Clean data

![find error](1.png) 

### Position Played
If you just read the data file with `df = pd.read_csv('aufgabe1.csv')` you will get error **"ParserError"**, since the data is somehow not "clean", which we need to deal with.
Concrete error: **ParserError: Error tokenizing data. C error: Expected 89 fields in line 3, saw 90**  
There should be 89 columns but some rows have 90, that's because in third 6th column "Positions Played" there are some players played more than one positions and in the data set use comma for &, e.g. 'CF,ST', but when it was read, this will cause confusion, because comma is to separate column.
What we can do is that replace , which is within '' with ;. We can use regex to identify this case, e.g. 'A,B', so we want 'A;B'

In [ ]:
import re # regex
import pandas as pd

# Specify input and output file paths
input_file = 'aufgabe1_ori.csv'
output_file = 'aufgabe1_clean.csv'

# Read the CSV file line by line and replace commas within single quotes
lines = []
with open(input_file, 'r') as file:
    for line in file:
        # Use regex to find commas within single quotes and replace them with a semicolon
        modified_line = re.sub(r"'(.*?)'", lambda x: x.group(0).replace(',', ';'), line)
        lines.append(modified_line)

# Write the modified lines to a new CSV file
with open(output_file, 'w') as file:
    file.writelines(lines)

### 1100000,00€ -> 1100000
Again Error --> ParserError: Error tokenizing data. C error: Expected 89 fields in line 12922, saw 90
![find error](2.png)
The reason is that the expression in german way: 1100000,00€. The comma here will also be translated to the separation of column. Because this is the sole case, so I'd like to change this manuelly in .csv file, e.g. 1100000,00€ --> 1100000. **Change it in aufgabe1_ori.csv!!!**

### Duplicates, unnecessary ''

In [ ]:
# Now read the cleaned data with pandas and save it in df
df = pd.read_csv(output_file)
# deal with duplicates and save it in df
df = df.drop_duplicates()
# '' is not necessary in for the columns name
df.columns = df.columns.str.strip("'")
# '' is also not necessary 
df = df.map(lambda x: x.strip("'") if isinstance(x, str) else x)
df

Using '' is to indicate this element is string, but here we use object as data type, should we change to string? maybe not, just leave it as object for now.  

### Data Type
DtypeWarning: Columns (4) have mixed types

when we use pd.read_csv to read the csv file we got DtypeWarning, it tells us that column 4 has mixed data types, but if we check the data types using df.dtypes, I didn't see mixed types. I'm not sure about this, just leave it for now, it seems to me not a problem to finish the task.

In [ ]:
# Check data types in column 4
unique_types = df['Value(in Euro)'].map(type).value_counts()
print(unique_types)

In [ ]:
# get data types of all columns
print(df.dtypes)
print(df.info())

In [ ]:
# Column: Value(in Euro)
# Convert 'Value(in Euro)' to int64 and replace NaN values with 0
df['Value(in Euro)'] = pd.to_numeric(df['Value(in Euro)'], errors='coerce')
df['Value(in Euro)'] = df['Value(in Euro)'].fillna(0).astype('int64')
# Column: Best Position
# 1. Convert 'Best Position' to string to reveal any hidden values
df['Best Position'] = df['Best Position'].astype(str)
# 2. Replace any "nan" strings (which represent the actual NaN values) with a placeholder, if desired
df['Best Position'] = df['Best Position'].replace("nan", "Unknown")  # or use a suitable placeholder
# Column: Positioning
df['Positioning'] = df['Positioning'].fillna(0).astype('int64')
# Column: Goalkeeper Handling
df['Goalkeeper Handling'] = df['Goalkeeper Handling'].fillna(0).astype('int64')


In [ ]:
print(df.info())

## Zweck der Daten

In [ ]:
print(df.columns)

In [ ]:
print(df.head())

In [ ]:
print(df.tail())

In [ ]:
print(df.describe())

## Analysis of the data purpose
This data set shows: (to be complete...)
- Information over soccer player, including basic personal information, analysis of skills, values etc.
- 

# Aufgabe 1b)-1d)
b) Visualisieren Sie die Daten aus a) mittels einer interaktiven Applikation in Python, indem Sie die Verteilungen darstellen:  
- Verteilung der Items
- Gegenüberstellung der Attribute Age und Wage (in Euro)
- Gegenüberstellung der Attribute Age und Overall  


c) Bauen Sie bei allen Darstellungen Interaktionsmöglichkeiten zum Filtern der Daten ein. <span style='color:#CC3333; background:; font-size:;'>Sollten die Daten Ausreißer haben, passen Sie Ihre Visualisierung dahingehend an.</span> Bauen Sie zudem Filtermöglichkeiten zum Filtern nach den Attribute Nationality und Club ein.  
d) Erstellen Sie eine Darstellung zum Vergleich von Datenpunkten, bsp. Zeile 5 mit Zeile 35 des Datensatz. Die zuvergleichenden Datenpunkte sollen interaktiv wählbar sein.

## Verteilung der Items Analysis

Verteilung hier ist mit Häufigkeitsverteilung gemeint... okay...

## Aufgabe App

In [ ]:
# pip install dash-ag-grid
import pandas as pd
from dash import Dash, html, dcc, Input, Output, callback
import plotly.express as px
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import dash_ag_grid as dag


# read data from the csv file
# df = pd.read_csv("aufgabe1_clean.csv")

# Initialize dash app with bootstrap theme
load_figure_template('morph')
app = Dash(__name__, external_stylesheets=[dbc.themes.MORPH])

fig = px.histogram(df, x='Age', y='Overall', histfunc='avg')

left_row_style = {'margin': '2px 0', 'textAlign' : 'right'}
row_style = {'margin': '2px 0'}
# sorted_nat = sorted(df['Nationality'].unique())

# Layout, HTML Components
app.layout = html.Div([
    
    # title
    html.H1(f'IVDA Praktikum Aufgabe 1', className='text-center pb-3'),
    
    # Dashboard
    # 1. Data in table
    html.H3(f'Data in table', className='text-center pb-3'),
    dag.AgGrid(
        id="enable-pagination",
        rowData=df.to_dict("records"),
        columnDefs=[{"field": i} for i in df.columns],
        defaultColDef={"filter": True},
        dashGridOptions={"pagination": True, "animateRows": False},
    ),

    # 2. Verteilung der Items
    html.H3(f'Verteilung der Items', className='text-center pb-3'),
    # dropdown for choosing attribute
    dcc.Dropdown(
        # options=['Age', 'Height(in cm)', 'Weight(in kg)', 'Overall', 'Potential', 'Nationality'],
        options=[{'label': col, 'value': col} for col in df.columns],
        value='Age', placeholder='Select Attribute (X-Axis)', id='select_attribute',
    ),
    dcc.Graph(figure=fig, id='verteilung'),

    # 3. Correlation between Overall and Age, Wage and Age
    html.H3(f'Correlation', className='text-center pb-3'),
    dcc.Dropdown(
        options=['Overall', 'Wage(in Euro)', 'Potential', 'Value(in Euro)'], 
        value='Overall', placeholder='Select Y-Axis', id='y_options',
    ),
    dcc.Graph(figure=fig, id='correlation'),

    # Filter Dropdown for nationality and club
    # ideal: if I don't choose --> not filter by that dimension/ if I choose "all"(if I can figure it out) --> all nationality or club
    html.Div([
        dcc.Dropdown(
            # options=[{'label': 'All', 'value':'All'}] + [{'label': nationality, 'value': nationality} for nationality in df['Nationality'].unique()], # sorted(df['Nationality'].unique())
            id='nationality',
            options=df['Nationality'].unique(),
            value='Germany',
            placeholder='Select Nationality',
            style={'width':'300px'},
        ),
        dcc.Dropdown(
            # options=[{'label': 'All', 'value':'All'}] + [{'label': club, 'value': club} for club in df['Club Name'].unique()],
            id='club',
            options=df['Club Name'].unique(),
            value='RB Leipzig',
            placeholder='Select Club',
            style={'width':'300px'},
        ),
    ],style={'display': 'flex',
            'justify-content': 'center',
            'align-items': 'center',
            'flex-direction': 'column',
            'width': '100%'}),
    
    # 3. comparasion between players
    html.Div([
        html.H3(f'Comparasion between players', className='text-center pb-3'),
        # choose two player and get image of two players
        dbc.Row([
            dbc.Col([],width=3, style={'textAlign':'center'}),
            dbc.Col([
                html.Div(
                    html.Img(id='player1_image',src='', style={'height':'120px'}), 
                        style={'border': '', 'padding': '10px'}
                ),
                dcc.Dropdown(
                    id='player1_dropdown', #this dropdown is every important in order to load image, basic info and all informations about this player
                    options=df['Full Name'].unique(),
                    value='Lionel Messi',
                    placeholder="Select Player 1"
                ),
            ], width=3, style={'textAlign':'center'}),
            dbc.Col([
                html.Div(
                    html.Img(id='player2_image', src='', style={'height':'120px'}), 
                            style={'border': '', 'padding': '10px'}),
                dcc.Dropdown(
                    id='player2_dropdown',
                    options=df['Full Name'].unique(),
                    value='Karim Benzema',
                    placeholder="Select Player 2"
                )
            ], width=3, style={'textAlign':'center'})
        ], justify="", className="mb-4"),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.P(f"Age", style=left_row_style),
                    html.P(f"Nationality", style=left_row_style),
                    html.P(f"Height", style=left_row_style),
                    html.P(f"Weight", style=left_row_style),
                    html.P(f"Club", style=left_row_style),
                    html.P(f"Best Position", style=left_row_style),
                    html.P(f"Preferred Foot", style=left_row_style),
                ])
            ], width=3, style={'textAlign':'center'}),
            # basic information of player1
            dbc.Col([
                html.Div(
                    id='player1_basic_information', 
                    style={'border': '', 'padding': '', 'width': '250px'})
            ], width=3, style={'textAlign':'center'}),
            # basic information of player2
            dbc.Col([
                html.Div(
                    id='player2_basic_information', 
                    style={'border': '', 'padding': '', 'width': '250px'})
            ], width=3, style={'textAlign':'center'}),
        ]),

    ]),

    # Footer
    html.Footer([
        html.Div([
            html.A('Author: Huo Jiang, Tina', href='https://github.com/hurryclear/interactive-visualization-data-analysis-app'),
            html.Span(' | '),
            html.A('Copyright', href='https://www.uni-leipzig.de')
        ], className='bg-light text-dark text-center py-3 fs-5')
    ])
])

# Section2: Verteilung
@callback(
    Output(component_id='verteilung', component_property='figure'),
    Input('select_attribute', 'value')
)
def update_verteilung(select_attribute):

    if pd.api.types.is_numeric_dtype(df[select_attribute]):
        # Ensure the 'Age' column is of an appropriate data type
        df[select_attribute] = pd.to_numeric(df[select_attribute], errors='coerce')
        # Create a histogram to show the absolute frequency of players by age
        fig = px.histogram(
            df, x=select_attribute, 
            nbins=len(df[select_attribute].unique()), 
            title="Absolute Frequency of Players by Age"
        )
    else:
        # For categorical attributes, use value counts
        category_counts = df[select_attribute].value_counts().reset_index()
        category_counts.columns = [select_attribute, 'Count']
        
        # Create a bar chart for categorical attributes
        fig = px.bar(category_counts, x=select_attribute, y='Count', title=f"Absolute Frequency of Players by {select_attribute}")

    # Update the layout to label the axes
    fig.update_layout(
        xaxis_title=select_attribute,
        yaxis_title="Absolute Frequency of Players",
        bargap=0.2
    )
    return fig


# Section3: Correlation
# function filter_data: filter data by certain attributes and return filtered data
def filter_data(df, nationality, club):
    df_filtered = df.copy()
    if nationality and nationality != '':
        df_filtered = df_filtered[df_filtered['Nationality'] == nationality]

    if club and club != '':
        df_filtered = df_filtered[df_filtered['Club Name'] == club]
    return df_filtered
# add controls to build the interaction
@callback(
    Output(component_id='correlation', component_property='figure'),
    Input(component_id='y_options', component_property='value'), # input for the callback functions, this corresponds to col_chosen
    Input(component_id='nationality', component_property='value'),
    Input(component_id='club', component_property='value'),
)
def update_graph(col_chosen, nationality, club): # club
    
    # by nationality or club
    df_filtered = filter_data(df, nationality, club)

    # If neither is selected or both are set to "All", df_filtered will remain unfiltered (all data)

    # px.histogram(df, x='Age', y=col_chosen, histfunc='avg')
    # not like histogram in line chart there is no embedded avg function, so we have to group the data by the age first and caculate the avg of overall and wage
    df_avg_by_age = df_filtered.groupby('Age', as_index=False)[col_chosen].mean()

    fig = px.line(df_avg_by_age, x = 'Age', y = col_chosen, title=f'Average of {col_chosen} by Age', markers=True)
    return fig # what you return will be assigned to the callback output component_property, in this case fig1 will be return to figure


# Section3: comparasion between players
# update information of player1
@callback(
    Output(component_id='player1_image', component_property='src'),
    # Output(component_id='', component_property=''),
    # Output(component_id='', component_property=''),
    Input(component_id='player1_dropdown', component_property='value'),
)
def update_image_player1(player1_name):
    if player1_name:
        image_link = df.loc[df['Full Name']==player1_name, 'Image Link'] #use loc to retrieve the Image Link column
        return image_link.iloc[0] if not image_link.empty else ''
    return ''

# update image of player2
@callback(
    Output(component_id='player2_image', component_property='src'),
    Input(component_id='player2_dropdown', component_property='value')
)
def update_image_player2(player2_name):
    if player2_name:
        image_link = df.loc[df['Full Name']==player2_name, 'Image Link']
        return image_link.iloc[0] if not image_link.empty else ''
    return ''

# update basic information of player1
@callback(
    Output('player1_basic_information', 'children'),
    Output('player2_basic_information', 'children'),
    Input('player1_dropdown','value'),
    Input('player2_dropdown','value')
)
def update_basic_player1(player1_name, player2_name):
    attributes_display1 = []
    attributes_display2 = []
    if player1_name:
        player1_info = df.loc[df['Full Name'] == player1_name].iloc[0]
        attributes_display1.append(html.P(f"{player1_info['Age']}", style=row_style))
        attributes_display1.append(html.P(f"{player1_info['Nationality']}", style=row_style))
        attributes_display1.append(html.P(f"{player1_info['Height(in cm)']}", style=row_style))
        attributes_display1.append(html.P(f"{player1_info['Weight(in kg)']}", style=row_style))
        attributes_display1.append(html.P(f"{player1_info['Club Name']}", style=row_style))
        attributes_display1.append(html.P(f"{player1_info['Best Position']}", style=row_style))
        attributes_display1.append(html.P(f"{player1_info['Preferred Foot']}", style=row_style))
    if player2_name:
        player2_info = df.loc[df['Full Name'] == player2_name].iloc[0]
        attributes_display2.append(html.P(f"{player2_info['Age']}", style=row_style))
        attributes_display2.append(html.P(f"{player2_info['Nationality']}", style=row_style))
        attributes_display2.append(html.P(f"{player2_info['Height(in cm)']}", style=row_style))
        attributes_display2.append(html.P(f"{player2_info['Weight(in kg)']}", style=row_style))
        attributes_display2.append(html.P(f"{player2_info['Club Name']}", style=row_style))
        attributes_display2.append(html.P(f"{player2_info['Best Position']}", style=row_style))
        attributes_display2.append(html.P(f"{player2_info['Preferred Foot']}", style=row_style))
    return attributes_display1, attributes_display2 

if __name__ == '__main__':
    app.run_server(debug=True)